In [1]:
import numpy as np
import matplotlib.pyplot as plt
import camb
from camb import model, initialpower
import andreap
from scipy.interpolate import interp1d

import importlib.util
import sys

In [2]:
test_index = 0
validation_set = True
# skips comparison with Andrea's code
fast = False

In [3]:
''' This messiness is necessary to import scripts which are not installed and
which are not located in the cwd.'''

spec = importlib.util.spec_from_file_location(
    "camb_interface", "../../cassL/camb_interface.py")
ci = importlib.util.module_from_spec(spec)
sys.modules["camb_interface"] = ci
spec.loader.exec_module(ci)

spec = importlib.util.spec_from_file_location(
    "generate_training_data", "../../cassL/generate_training_data.py")
gtd = importlib.util.module_from_spec(spec)
sys.modules["generate_training_data"] = gtd
spec.loader.exec_module(gtd)

In [4]:
hc = np.load("hc_G3_redo.npy",
    allow_pickle=True)
samples = np.load("samples_G3_redo.npy",
    allow_pickle=True)

if False:
    hc = np.load("hc_backup_i4999_massive_pred_revised.npy",
        allow_pickle=True)
    samples = np.load("samples_backup_i4999_massive_pred_revised.npy",
        allow_pickle=True)

if validation_set:
    hc = np.load("hc_test_G3.npy",
        allow_pickle=True)
    samples = np.load("samples_test_G3.npy",
        allow_pickle=True)
    
standard_k = np.load("standard_k.npy", allow_pickle=True)

In [5]:
test_cosmology = hc[test_index]
test_pspec = samples[test_index]

In [6]:
ombh2 = test_cosmology[0]
omch2 = test_cosmology[1]
ns = test_cosmology[2]
sigma12 = test_cosmology[3]
As = test_cosmology[4]
omnuh2 = test_cosmology[5]

In [7]:
full_cosm = gtd.build_cosmology(ombh2, omch2, ns, sigma12, As, omnuh2)

In [8]:
mnu = full_cosm['mnu']

In [9]:
if not fast:
    Pk_lo_h = andreap.get_PK(ombh2, omch2, ns, mnu, 10, As, w_mzero=True)
    Pk_hi_h = andreap.get_PK(ombh2, omch2, ns, mnu, 67, As, w_mzero=True)
    Pk_target_h = andreap.get_PK(ombh2, omch2, ns, mnu, 57, As, w_mzero=True)

0.008193
1
massive-neutrino sigma12 [1.21363354]
massless-neutrino sigma12 [1.57252607]
0.008193
1
massive-neutrino sigma12 [0.67411464]
massless-neutrino sigma12 [0.86873491]
0.008193
1
massive-neutrino sigma12 [0.72529371]
massless-neutrino sigma12 [0.93539084]


In [10]:
if not fast:
    z = np.linspace(0., 5., 500)
    sig12 = np.zeros(500)
    for i in range(500):
        sig12[i] = andreap.get_s12(Pk_hi_h['mzero'], z[i])

    sig12[0]
    
    sig12_interpolator = interp1d(sig12, z, kind='cubic')

In [11]:
sigma12

0.7850399905264863

In [12]:
g_pspec, g_sigma12, g_z = gtd.evaluate_cell(full_cosm, standard_k)
print("This is our approximation", g_sigma12, "at redshift", g_z)

This is our approximation 0.785039985478845 at redshift 0.20376232047305143


In [13]:
import copy as cp
tilde_cosm = cp.deepcopy(full_cosm)
tilde_cosm['omch2'] += tilde_cosm['omnuh2']
tilde_cosm = ci.specify_neutrino_mass(tilde_cosm, 0, 0)
gtd.print_cosmology(tilde_cosm)
print()
gtd.print_cosmology(full_cosm)

ombh2 0.021058815
omch2 0.10322400000000001
n_s 0.9328430000000001
A_s 2.8660491636915983e-09
OmK 0.0
OmL 0.681415906
h 0.67
w0 -1.0
wa 0.00
sigma12 0.7850399905264863
omnuh2 0
mnu 0.0

ombh2 0.021058815
omch2 0.095031
n_s 0.9328430000000001
A_s 2.8660491636915983e-09
OmK 0.0
OmL 0.681415906
h 0.67
w0 -1.0
wa 0.00
sigma12 0.7850399905264863
omnuh2 0.008193
mnu 0.7619760361638409


In [15]:
kci_t, zci_t, pci_t, sci_t = ci.evaluate_cosmology(tilde_cosm, redshifts=[g_z], k_points=300)
kci, zci, pci, sci = ci.evaluate_cosmology(full_cosm, redshifts=[g_z], k_points=300)

In [16]:
sci

array([0.60973944])

In [17]:
g_pspec_t, g_sigma12_t, g_z_t = gtd.evaluate_cell(tilde_cosm, standard_k)
print("This is our approximation", g_sigma12_t)

AttributeError: module 'generate_training_data' has no attribute 'psz'

In [ ]:
if not fast:
    andreap.get_s12(Pk_hi_h['mzero'], 0)

In [ ]:
# I cannot figure out how to get AndreaP's function to work
#andreap.get_s12_fixedz(Pk_hi_h['mzero'])

In [ ]:
tilde_cosm

In [ ]:
plt.plot(standard_k, (pci - samples[test_index]) / samples[test_index] * 100)
plt.xscale('log')
plt.title("Discrepancy between Saved Spectrum and Direct kzps Call")
plt.ylabel('Percent Error')
plt.xlabel('scale $k$ [1 / Mpc]')

In [ ]:
plt.plot(standard_k, (g_pspec - samples[test_index]) / samples[test_index] * 100)
plt.xscale('log')
plt.title("Discrepancy between Saved Spectrum and On-the-fly psz Call")
plt.ylabel('Percent Error')
plt.xlabel('scale $k$ [1 / Mpc]')

In [ ]:
plt.plot(standard_k, (pci - g_pspec) / pci * 100)
plt.xscale('log')
plt.title("Discrepancy between On-the-fly psz and kzps Calls")
plt.ylabel('Percent Error')
plt.xlabel('scale $k$ [1 / Mpc]')

This most recent plot is weak enough that we're happy. But why do we see differences in the previous plots? Is this a reproduceable discrepancy?

In [ ]:
resample, bestz = gtd.fill_hypercube(hc, standard_k, massive_neutrinos=True,
    cell_range=range(1))
resample = resample[0]

In [ ]:
plt.plot(standard_k,
    (resample - samples[test_index]) / samples[test_index] * 100)
plt.xscale('log')